## Setting Up

In [2]:
import os

MODEL = os.getenv("OPENAI_MODEL")

In [3]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=MODEL
)

## Agent Runner

Running the Agent Asynchronously

`Runner.run()` runs the agent in async mode

In [5]:
result = await Runner.run(
    agent,
    "Tell me a short story"
)

result.final_output

'Once upon a time in a small village nestled between rolling hills, there lived an elderly woman named Elara. She was known for her vibrant garden, which blossomed with every flower imaginable. Each spring, her garden would awaken, painting the landscape with bursts of color and filling the air with delightful fragrances.\n\nOne day, a young girl named Mia, curious and adventurous, wandered into Elara’s garden. Mesmerized by its beauty, she walked carefully among the flowers, trying to absorb every detail. Elara, noticing the girl’s wonder, approached her with a warm smile.\n\n“Would you like to help me plant some new seeds?” she asked.\n\nMia’s eyes lit up. “Yes, please! What kind of seeds?”\n\nElara handed her a packet labeled "Moonlight Blossoms." “These only bloom at night and glow softly like stars,” she explained. “They remind us that even in darkness, there is beauty.”\n\nTogether, they dug small holes and planted the seeds, talking and laughing as they worked. Elara shared stor

### Async Streaming Mode

Start by creating a response using `Runner.run_streamed(...)`  \
Then iterate over it asynchronously

In [ ]:
response = Runner.run_streamed(
    agent,
    "Tell me a short story"
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='resp_67ed7db490448191a7f60242bc1dd76b099562524aee8de2', created_at=1743617460.0, error=None, incomplete_details=None, instructions="You're a helpful assistant", metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[], parallel_tool_calls=False, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reaso

## Events

The streaming response returns **Events**. This helps us filter the output to extract relevant information.

There are several events:
* RawEvent: Token output
* AgentUpdated: Agent creation
* StreamEvent: A message being completed
* 

In [ ]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    agent,
    "Tell me a short story"
)

async for event in response.stream_events():
     if event.type == "raw_response_event" and \
         isinstance(event.data, ResponseTextDeltaEvent):
             print(event.data.delta, end="", flush=True)

Once upon a time in a quaint village, tucked between rolling hills, lived a curious little fox named Finn. Unlike other foxes who prowled the woods, Finn was enchanted by the stars. Each night, he would climb to the highest hill and gaze up at the sparkling sky, dreaming of adventures beyond his forest.

One evening, as Finn lay among the soft grass, he spotted a twinkling star that seemed to beckon him. Intrigued, he followed a winding path deeper into the woods. To his surprise, he stumbled upon a hidden glade where the air was thick with magic. In the center stood an ancient tree, its branches shimmering with starlight.

"Welcome, Finn," said the tree with a voice like rustling leaves. "You have come to seek your adventure."

Finn's eyes widened in awe. "Can you send me among the stars?"

With a gentle sway, the tree answered, "Close your eyes and believe. Your heart will guide you."

Taking a deep breath, Finn did as instructed. Suddenly, he felt weightless, and when he opened his 

## Tools

In [15]:
from agents import function_tool

@function_tool
def multiply(x: int, y: int) -> int:
    """Multiply 2 numbers to provide the product"""
    return x*y

In [18]:
agent = Agent(
    name="Assistant",
    instructions="""
        You're a helpful assistant who uses,
        provided tools to help whenever possible.,
        Do not rely on your own knowledge too much,
        instead use the tools to answer queries.
    """,
    model=MODEL,
    tools=[multiply]
)

In [19]:
response = Runner.run_streamed(
    agent,
    "What is the product of 4 and 100 ?"
)

async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="\n        You're a helpful assistant who uses,\n        provided tools to help whenever possible.,\n        Do not rely on your own knowledge too much,\n        instead use the tools to answer queries.\n    ", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='multiply', description='Multiply 2 numbers to provide the product', params_json_schema={'properties': {'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}, 'required': ['x', 'y'], 'title': 'multiply_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x779cac1f3400>, strict_json_schema=True)], mcp_servers=[], inpu

### Parsing Tool calling Events

In [20]:
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,
    ResponseTextDeltaEvent
)

response = Runner.run_streamed(
    starting_agent=agent,
    input="Multiply 4 with 7"
)

async for event in response.stream_events():
    
    if event.type == 'raw_response_event':
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            # tool call
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            # final answer
            print(event.data.delta, end="", flush=True)
    elif event.type == 'agent_updated_stream_event':
        # current agent
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == 'run_item_stream_event':
        if event.name == 'tool_called':
            # this is complete collection of tool call after the tokens indicating
            # tool call are completely streamed out
            print()
            print(f"> Tool called: {event.item.raw_item.name}")
            print(f"> Tool args: {event.item.raw_item.arguments}")
        elif event.name == 'tool_output':
            print(f"> Tool Output: {event.item.raw_item['output']}")
            
            

> Current Agent: Assistant
{"x":4,"y":7}
> Tool called: multiply
> Tool args: {"x":4,"y":7}
> Tool Output: 28
The product of 4 and 7 is 28.

## Guardrails

In [21]:
from pydantic import BaseModel

# define structure for output guardrail
class GuardrailOutput(BaseModel):
    is_triggered: bool
    reasoning: str
    
# an agent that checks if user is asking about political opinions
politics_agent = Agent(
    name="Politics check",
    instructions="Check if the user is asking you about political opinions.",
    output_type=GuardrailOutput
)

In [22]:
query = "What do you think about BJP in India ?"

result = await Runner.run(starting_agent=politics_agent, input=query)
result

RunResult(input='What do you think about BJP in India ?', new_items=[MessageOutputItem(agent=Agent(name='Politics check', instructions='Check if the user is asking you about political opinions.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=<class '__main__.GuardrailOutput'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_67ee62440e98819182d35491bc6f82560db59507b316d3da', content=[ResponseOutputText(annotations=[], text='{"is_triggered":true,"reasoning":"The user is asking for an opinion about a political party, BJP in India. This falls under the category of political opinions, which the prompt identifies as sensitive content."}', type='output_text')], ro

In [23]:
result.final_output

GuardrailOutput(is_triggered=True, reasoning='The user is asking for an opinion about a political party, BJP in India. This falls under the category of political opinions, which the prompt identifies as sensitive content.')